In [2]:
import torch
import torch.nn as nn

In [4]:
vector_dimension = 768 

linear_dimension = 3072 # dimension of the linear layer of feed forward for normalization

image_size = 224 # pixels in each dimension of image

num_channels = 3 # number of channels in image (R, G, B)

patch_size = 16 # number of patches/ blocks we break each image into

attention_heads = 12 # number of attention heads

num_layers = 12 # number of transformer block layers in the model

dropout = 0.4

normalization_constant = 1e-6 # proportionality constant for normalization layer

num_image_tokens : int = None # number of tokens produced for each image i.e. it produces a list of vectors/ embeddings for a patch of each image

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

cuda


In [ ]:
class VisionTransformer(nn.Module):

    def __init__(self, vector_dimension, normalization_constant):
        super().__init__()

        self.embedding_layer

        self.encoder_layer

        self.post_normalization_layer = nn.LayerNorm(vector_dimension, normalization_constant)

    def forward(self, image_data):

        hidden_state = self.embedding_layer(image_data)

        hidden_state = self.encoder_layer(hidden_state)

        hidden_state = self.post_normalization_layer(hidden_state)

        return hidden_state

NameError: name 'nn' is not defined